In [1]:
import numpy as np
import matplotlib.pyplot as plt
import options_trading as opt
%load_ext autoreload
%autoreload 2

### Price a Digit Option

In [3]:
def call_spread(S0, K1, K2, T, r, vol):
    C1 = opt.black_scholes(r, S0, K1, T, vol, type="c")
    C2 = opt.black_scholes(r, S0, K2, T, vol, type="c")
    return C1 - C2

def put_spread(S0, K1, K2, T, r, vol):
    P1 = opt.black_scholes(r, S0, K1, T, vol, type="p")
    P2 = opt.black_scholes(r, S0, K2, T, vol, type="p")
    return P2 - P1

In [ ]:
def digit_option_call_spread(A, S0, K, T, w, r, vol, opttype="C"):
    """
    A : montant du paiement de la digital option
    S0 : prix de l'actif sous-jacent à t=0
    K : strike de la digital option
    T : maturité (en années)
    w : largeur du spread (> 0)
    r : taux sans risque (en pourcentage)
    vol : volatilité (en pourcentage)
    opttype : "C" pour call, "P" pour put
    """
    
    if w <= 0:
        raise ValueError("The width of the spread must be greater than 0.")
    
    if opttype == "C":
        ### Buy A/w call spreads (gearing of the digit option call)
        return (A / w) * call_spread(S0, K - w, K, T, r, vol)
    
    elif opttype == "P":
        ### Buy A/w put spreads (gearing of the digit option put)
        return (A / w) * (opt.black_scholes(r, S0, K, T, vol, type="p") -
                          opt.black_scholes(r, S0, K - w, T, vol, type="p"))

### Hedge a Digit Option

In [ ]:
def delta_call_spread(S0, K1, K2, T, r, vol):
    delta_C1 = opt.delta_calc(r, S0, K1, T, vol, type="c")
    delta_C2 = opt.delta_calc(r, S0, K2, T, vol, type="c")
    return delta_C1 - delta_C2

def delta_put_spread(S0, K1, K2, T, r, vol):
    delta_P1 = opt.delta_calc(r, S0, K1, T, vol, type="p")
    delta_P2 = opt.delta_calc(r, S0, K2, T, vol, type="p")
    return delta_P2 - delta_P1

def gamma_call_spread(S0, K1, K2, T, r, vol):
    gamma_C1 = opt.gamma_calc(r, S0, K1, T, vol, type="c")
    gamma_C2 = opt.gamma_calc(r, S0, K2, T, vol, type="c")
    return gamma_C1 - gamma_C2

def gamma_put_spread(S0, K1, K2, T, r, vol):
    gamma_P1 = opt.gamma_calc(r, S0, K1, T, vol, type="p")
    gamma_P2 = opt.gamma_calc(r, S0, K2, T, vol, type="p")
    return gamma_P2 - gamma_P1

In [ ]:
def hedging_digit_option(S0, K, T, w, r, vol, N, M, opttype="C"):
    """
    S0 : price of the underlying at time t=0
    K : strike of the digit
    T : maturity (in years)
    w : width of the call spread (must be > 0)
    r : risk-free rate (in percent)
    vol : volatility (in percent)
    N : number of timesteps
    M : number of simulations
    opttype : "C" for call option and "P" for put option
    """

    dt = T / N
    nudt = (r - 0.5 * vol ** 2) * dt
    volsdt = vol * np.sqrt(dt)
    
    hedged_position_avg = np.zeros(N+1)
    cash_position_avg = np.zeros(N+1)
    time = np.linspace(0, T, N+1)

    # Simulation Monte Carlo
    for i in range(M):
        St = S0
        hedged_position = np.zeros(N+1)
        cash_position = np.zeros(N+1)
        
        # Calcul du delta initial
        if opttype == "C":
            delta_initial = delta_call_spread(S0, K - w, K, T, r, vol)
            gamma_initial = gamma_call_spread(S0, K - w, K, T, r, vol)
        else:
            delta_initial = delta_put_spread(S0, K - w, K, T, r, vol)
            gamma_initial = gamma_put_spread(S0, K - w, K, T, r, vol)
        
        hedged_position[0] = delta_initial

        for j in range(1, N+1):
            epsilon = np.random.normal()
            St = St * np.exp(nudt + volsdt * epsilon)
            T_remaining = T - time[j]

            if opttype == "C":
                new_delta = delta_call_spread(St, K - w, K, T_remaining, r, vol)
                new_gamma = gamma_call_spread(St, K - w, K, T_remaining, r, vol)
            else:
                new_delta = delta_put_spread(St, K - w, K, T_remaining, r, vol)
                new_gamma = gamma_put_spread(St, K - w, K, T_remaining, r, vol)

            delta_change = new_delta - hedged_position[j-1]

            if abs(new_gamma) > 0.1:
                delta_change *= (1 + abs(new_gamma))

            hedged_position[j] = new_delta

            cash_position[j] = cash_position[j-1] + delta_change * St

        hedged_position_avg += hedged_position
        cash_position_avg += cash_position

    hedged_position_avg /= M
    cash_position_avg /= M

    return hedged_position_avg, cash_position_avg, time